<a href="https://colab.research.google.com/github/tmss1212/backtesting/blob/main/backtesting_FX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#ライブラリインストール
!pip install backtesting numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=23e256681bc24ed3916496dd3018b5a72bb7958989c4a9139583ba920903aec3
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [12]:
#データ読み込み
import pandas as pd
import datetime as dt
import numpy as np
import pandas_datareader.data as web

csv_path = "/content/drive/MyDrive/FX/GBPJPY60.csv"
cols = ('Date', 'Time', 'Open', 'Low', 'High', 'Close', 'Volume')
df = pd.read_csv(csv_path, names=cols)
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.set_index('DateTime')
df = df.drop(['Date','Time'], axis=1)

print("======= 生データ =======")
print(df.head(3))
print(df.tail(3))

start = '2023-10-01'
end = '2023-12-08'
df = df.loc[start:end]

print(" ")
print("======= 期間抽出 =======")
print(df.head(3))
print(df.tail(3))

======= 生データ =======
                        Open      Low     High    Close  Volume
DateTime                                                       
2023-08-11 16:00:00  183.672  183.983  183.615  183.933   11572
2023-08-11 17:00:00  183.933  184.231  183.575  183.974   13083
2023-08-11 18:00:00  183.979  184.083  183.849  183.964    8615
                        Open      Low     High    Close  Volume
DateTime                                                       
2023-12-08 21:00:00  181.894  181.927  181.801  181.917    4999
2023-12-08 22:00:00  181.919  181.937  181.803  181.911    4308
2023-12-08 23:00:00  181.912  181.931  181.804  181.860    3656
 
======= 期間抽出 =======
                        Open      Low     High    Close  Volume
DateTime                                                       
2023-10-02 00:00:00  182.444  182.455  182.250  182.264    1583
2023-10-02 01:00:00  182.244  182.423  182.083  182.225    4082
2023-10-02 02:00:00  182.229  182.380  182.212  182.305    2

In [13]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
#トレード戦略クラス
class SmaCross(Strategy):
    #短期取得日数
    n1 = 10
    #長期取得日数
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma_short = self.I(SMA, close, self.n1)
        self.sma_long = self.I(SMA, close, self.n2)

    #売買条件アルゴリズム
    def next(self):
        #ゴールデンクロス
        if crossover(self.sma_short, self.sma_long):
            self.buy()
        #デッドクロス
        elif crossover(self.sma_long, self.sma_short):
            self.sell()

#トレード戦略クラス
bt = Backtest(data=df,
              strategy=SmaCross,
              cash=3000000,
              commission=.002,
              exclusive_orders=True)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [14]:
#バックテスト実行
output = bt.run()
print(output)
bt.plot()

Start                     2023-10-02 00:00:00
End                       2023-12-08 23:00:00
Duration                     67 days 23:00:00
Exposure Time [%]                        95.0
Equity Final [$]               2764491.987372
Equity Peak [$]                3026921.892024
Return [%]                          -7.850267
Buy & Hold Return [%]               -0.221656
Return (Ann.) [%]                  -33.233442
Volatility (Ann.) [%]                6.259268
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.371407
Avg. Drawdown [%]                   -2.046968
Max. Drawdown Duration       57 days 18:00:00
Avg. Drawdown Duration       10 days 22:00:00
# Trades                                   59
Win Rate [%]                        22.033898
Best Trade [%]                       1.766348
Worst Trade [%]                     -0.824375
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1309', ...)

In [16]:
#パラメータ最適化後
output_opt = bt.optimize(n1=range(1, 24, 1),
                         n2=range(1, 24, 1),
                         constraint=lambda p: p.n1 < p.n2,
                         maximize='Return [%]',
                         method='grid',
                         max_tries=None,
                         return_heatmap=False,
                         return_optimization=False,
                         random_state=None)
print(output_opt)
bt.plot()

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

Start                     2023-10-02 00:00:00
End                       2023-12-08 23:00:00
Duration                     67 days 23:00:00
Exposure Time [%]                   94.833333
Equity Final [$]               2874879.579076
Equity Peak [$]                3038659.912684
Return [%]                          -4.170681
Buy & Hold Return [%]               -0.221656
Return (Ann.) [%]                  -18.982219
Volatility (Ann.) [%]                7.866064
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -7.064809
Avg. Drawdown [%]                   -1.009624
Max. Drawdown Duration       56 days 03:00:00
Avg. Drawdown Duration        6 days 13:00:00
# Trades                                   49
Win Rate [%]                        24.489796
Best Trade [%]                       1.700584
Worst Trade [%]                     -0.697432
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1989', ...)